In [1]:
import torch

from collections import deque

import numpy as np

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import gym

In [2]:
import wandb
wandb.login()
run=wandb.init(project="lab4", entity="ieor-4575", tags=["torch"])

wandb: Currently logged in as: ieor-4575 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.24 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


## Policy Gradient

In the previous lab, we talked about value based method for reinforcement learning. In this lab, we focus on policy based method.

In policy based methods, intead of defining a value function $Q_\theta(s,a)$ and inducing a policy based on argmax, we parameterize a stochastic policy directly. The policy is parameterized as a categorical distribution over actions. Let it be $\pi_\phi(s)$ with parameter $\phi$, then the policy is defined by sampling actions $$a \sim \pi_\phi(s)$$

The policy induces a probability $p(\tau)$ over trajectories $\tau = \{s_0,a_0,s_1,a_1,..\}$. The expected total discounted reward is 

$$\rho(\phi) = \mathbb{E}_{\tau \sim p(\tau)} \big[R(\tau)\big] = \mathbb{E}_{\pi_\phi} \big[\sum_{t=0}^\infty r_t \gamma^t \big]$$

The aim is to find $\phi$ such that the expected reward induced by $\pi_\phi$ is maximized.

### Policy Gradient Computation

We can derive policy gradient

$$\nabla_\phi \rho(\phi) = \mathbb{E}_{\pi} \big[\sum_{t=0}^\infty Q^{\pi_\phi}(s_t,a_t) \nabla_\phi \log \pi_\phi(s_t, a_t) \big]$$

To compute the gradient for update $\phi \leftarrow \phi + \alpha \nabla_\phi \rho(\phi)$, we need to estimate $Q^{\pi_\phi}(s,a)$. Since $Q^{\pi_\phi}(s,a)$ is usually not analytically accessible, it can be approximated by 
1. Monte Carlo estimate
2. Train a value function $Q_\theta(s,a) \approx Q^{\pi_\phi}(s,a)$ and use it as a proxy
3. Mixture of both above

Before estimating $Q^{\pi_\phi}(s,a)$, let us write a parameterized policy over actions. The policy $\pi_\phi(s)$ takes a state as input and outputs a categorical distribution over actions. For example, if we have two actions, the probability vector to output is of the form $\pi_\phi(s)=[0.6,0.4]$.

**Loss function**
Given samples of state action pairs $(s_i, a_i)$ and estimate $Q_i$ for $Q^{\pi_\phi}(s_i, a_i)$, for $i=1,\ldots, $ we  set the loss function  as 
$$-\frac{1}{N} \sum_{i=1}^N Q_i \log(\pi_\phi(s_i, a_i)) $$
The loss function enables us  to compute policy gradients in implementation. The negative gradient of the above loss function has the form

$$\frac{1}{N} \sum_{i=1}^N Q_i  \nabla_\phi \log \pi_\phi(s_i, a_i) $$

where $Q_i$s are estimated and $\nabla_\phi \log\pi_\phi(s_i, a_i)$s are computed via backprop.

In [3]:
# define neural net \pi_\phi(s) as a class

class Policy(object):
    
    def __init__(self, obssize, actsize, lr):
        """
        obssize: size of the states
        actsize: size of the actions
        """
        # TODO DEFINE THE MODEL
        self.model = torch.nn.Sequential(
                    #input layer of input size obssize
                    torch.nn.Linear(obssize, 256),
                    #intermediate layers
                    torch.nn.ReLU(),
                    torch.nn.Linear(256, actsize)
                    #output layer of output size actsize
                )
        
        # DEFINE THE OPTIMIZER
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.99)
        
        # RECORD HYPER-PARAMS
        self.obssize = obssize
        self.actsize = actsize
        
        # TEST
        self.compute_prob(np.random.randn(obssize).reshape(1, -1))
    
    def compute_prob(self, states):
        """
        compute prob distribution over all actions given state: pi(s)
        states: numpy array of size [numsamples, obssize]
        return: numpy array of size [numsamples, actsize]
        """
        states = torch.FloatTensor(states)
        prob = torch.nn.functional.softmax(self.model(states), dim=-1)
        return prob.cpu().data.numpy()

    def _to_one_hot(self, y, num_classes):
        """
        convert an integer vector y into one-hot representation
        """
        scatter_dim = len(y.size())
        y_tensor = y.view(*y.size(), -1)
        zeros = torch.zeros(*y.size(), num_classes, dtype=y.dtype)
        return zeros.scatter(scatter_dim, y_tensor, 1)
    
    def train(self, states, actions, Qs):
        """
        states: numpy array (states)
        actions: numpy array (actions)
        Qs: numpy array (Q values)
        """
        states = torch.FloatTensor(states)
        actions = torch.LongTensor(actions)
        Qs = torch.FloatTensor(Qs)
        
        # COMPUTE probability vector pi(s) for all s in states
        logits = self.model(states)
        prob = torch.nn.functional.softmax(logits, dim=-1)

        # Compute probaility pi(s,a) for all s,a
        action_onehot = self._to_one_hot(actions, actsize)
        prob_selected = torch.sum(prob * action_onehot, axis=-1)
        
        # FOR ROBUSTNESS
        prob_selected += 1e-8

        # TODO define loss function as described in the text above
        # loss = ....
        
        loss = - torch.mean(Qs * torch.log(prob_selected))
        
        # BACKWARD PASS
        self.optimizer.zero_grad()
        loss.backward()

        # UPDATE
        self.optimizer.step()
        self.scheduler.step()
            
        return loss.detach().cpu().data.numpy()

Try to rollout trajecories using the policy

In [4]:
# Below is a set of template code for running a policy to interact with the environment
# It initializes a policy and runs it
# Note that you may not be able to run the code properly if there still some undefined components on the Policy class

env = gym.make("CartPole-v0")
obssize = env.observation_space.low.size
actsize = env.action_space.n
policyinit=Policy(obssize, actsize, 0.1)

obs = env.reset()
done = False
while not done:
    prob = policyinit.compute_prob(np.expand_dims(obs,0))
    prob /= np.sum(prob) #normalizing again to account for numerical errors
    action = np.asscalar(np.random.choice(actsize, p=prob.flatten(), size=1)) #choose according distribution prob
    obs, reward, done, info = env.step(action)

/Users/syeehyn/opt/anaconda3/envs/ieor4575/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  from ipykernel import kernelapp as app


### Estimate $Q^\pi(s,a)$

To estimate $Q^\pi(s,a)$, we can rollout the policy until the episode ends and do monte carlo estimate. In particular, under policy $\pi$, we start from state action $(s_0,a_0)$ and rollout the policy to generate a trajectory $\{s_0,a_0,s_1,a_1...s_T,a_T\}$, with corresponding reward $r_0,r_1...r_T$. Monte carlo estimate is 

$$\hat{Q}_{MC}(s,a) = \sum_{t=0}^T r_t \gamma^t \approx Q^\pi(s,a)$$

This estimate by itself is of high variance. Using pure monte carlo estimate may work but the gradient can have large variance and hence take the algorithm  a long time to converge. We can reduce variance using baseline. Recall the derivation of PG

$$\nabla_\phi \rho(\phi) = \mathbb{E}_{\pi_\phi} \big[\sum_{t=0}^\infty Q^{\pi_\phi}(s_t,a_t) \nabla_\phi \log \pi_\phi(s_t, a_t) \big] = \mathbb{E}_{\pi_\phi} \big[\sum_{t=0}^\infty ( Q^{\pi_\phi}(s_t,a_t) - b(s_t)) \nabla_\phi \log \pi_\phi(s_t, a_t) \big]$$

where $b(s_t)$ can be any function of state $s_t$. $b(s_t)$ is called baseline. Optimal baseline function is hard to compute, but a good proxy is the value function $V^{\pi_\phi}(s_t)$. Hence the gradient has the form 
$$\nabla_\phi \rho(\pi_{\phi}) = \mathbb{E}_{\pi} \big[\sum_{t=0}^\infty A^{\pi_\phi}(s_t,a_t) \nabla_\phi \log \pi_\phi(s_t, a_t) \big]$$

where $A^{\pi_\phi}(s,a)$ is the advantage. Hence we can train a value function $V^{\pi_\phi}(s)$ along side the policy and use it as baseline to reduce the variance of PG. 

Hence we also parameterize a value function $V_\theta(s) \approx V^{\pi_\phi}(s)$ with parameter $\theta$ to serve as baseline. The function takes as input the states $s$ and outputs a real value. 

Notice that unlike DQN, where $Q_\theta(s,a) \approx Q^\ast(s,a)$, now we have $V_\theta(s) \approx V^{\pi_\phi}(s)$. Therefore, we have a moving target to approximate, that changes with the current policy $\pi_\phi$. As $\phi$ is updated by PG, $\pi_\phi$ keeps changing, which changes $V^{\pi_\phi}(s)$ as well. We need to adapt $V_\theta(s)$ online to cater for the change in policy. 

Recall that to evaluate a policy $\pi$, we collect rollouts using $\pi$. If we start with state $s_0$, the reward following $\pi$ thereafter is $r_0...r_{T}$  then 

$$V^\pi(s_0) \approx \sum_{t=0}^{T} r_t \gamma^{t} = \hat{V}(s_0)$$

In general, given a trajectory $(s_0, a_0, s_1, a_1, r_1, s_2, a_2, r_2, ..., s_{T+1})$

$$\hat{V}(s_i) = \sum_{i=t}^{T} r_i \gamma^{i-t}$$

And the objective to minimize over is 
$$\frac{1}{T+1} \sum_{i=0}^{T} (V_\theta(s_i) - \hat{V}(s_i))^2$$

Since the policy keeps updating, we do not need to minimize the above objective to optimality. In practice, taking one gradient step w.r.t. above objective suffices.

In the code cell below, define the neural network to learn value function estimate. The implementation is similar to Qfunction class in lab3, except that inputs are only states, and not actions.

In [5]:
# TODO: define value function as a class. You need to define the model and set the loss.

class ValueFunction(object):
    
    def __init__(self, obssize, lr):
        """
        obssize: size of states
        """
        # TODO DEFINE THE MODEL
        self.model = torch.nn.Sequential(
                    #TODO
                      #input layer of input size obssize
                        torch.nn.Linear(obssize, 256),
                      #intermediate layers
                        torch.nn.ReLU(),
                      #output layer of output size 1
                        torch.nn.Linear(256, 1)
                )
        
        # DEFINE THE OPTIMIZER
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.99)
        
        # RECORD HYPER-PARAMS
        self.obssize = obssize
        self.actsize = actsize
        
        # TEST
        self.compute_values(np.random.randn(obssize).reshape(1, -1))
    
    def compute_values(self, states):
        """
        compute value function for given states
        states: numpy array of size [numsamples, obssize]
        return: numpy array of size [numsamples]
        """
        states = torch.FloatTensor(states)
        return self.model(states).cpu().data.numpy()
    
    def train(self, states, targets):
        """
        states: numpy array
        targets: numpy array
        """
        states = torch.FloatTensor(states)
        targets = torch.FloatTensor(targets)
        
        # COMPUTE Value PREDICTIONS for states 
        v_preds = self.model(states)

        # LOSS
        # TODO: set LOSS as square error of predicted values compared to targets
        #loss = 
        loss = torch.mean(torch.square(v_preds - targets))
        # BACKWARD PASS
        self.optimizer.zero_grad()
        loss.backward()
        # UPDATE
        self.optimizer.step()
        self.scheduler.step()
        return loss.detach().cpu().data.numpy()

### Summary of pseudocode

The critical components of the pseudocode are as follows.

**Collect trajectories** Given current policy $\pi_\phi$, we can rollout using the policy by executing $a_t \sim \pi_\phi(s_t)$. 

**Update value function** Value function update is based on minimizing the L2 loss between predicted value function and estimated value functions. For each state $s_i, i=0,\ldots, T$ in a trajectory of length $T+1$, compute $\hat{V}(s_i)$ as dicounted reward over the rest of the path (as defined above).  
Then take one gradient step to update $\theta$ using the gradient of the following loss: 

$$\frac{1}{T+1} \sum_{i=0}^T(V_\theta(s_i) - \hat{V}(s_i))^2$$

For your conveience, below we have provided a function discounted_rewards(r,gamma) that takes as inputs a list of $T$ rewards $r$ and computes all discounted rewards $\hat V(s_i), i=0, 1, 2, \ldots, T$. 

**Update policy using PG** To compute PG, we need to first monte carlo estimate action-value function $\hat{Q}(s_i,a_i)$. Given a trajectory with rewards $r=[r_0, r_1, r_2, \ldots, r_T]$, this can also be computed for all $s_i, a_i$ in this trajectory using the discounted_rewards(r, gamma) function below.

Then use value function as a baseline to compute advantage

$$\hat{A}(s_i,a_i) = \hat{Q}(s_i,a_i) - V_\theta(s_i)$$

Then compute surrogate loss 

$$L = - \frac{1}{(T+1)}\sum_{i} \hat{A}(s_i,a_i) \log \pi(a_i|s_i) $$

The policy is updated by $$\phi \leftarrow \phi - \alpha  \nabla_\phi L \approx \phi + \alpha \nabla_\phi \rho(\pi_\phi)$$

In [6]:
def discounted_rewards(r, gamma):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_sum = 0
    for i in reversed(range(0,len(r))):
        discounted_r[i] = running_sum * gamma + r[i]
        running_sum = discounted_r[i]
    return list(discounted_r)

### Main implementation : Policy gradient algorithm

Combine all the above steps and implement the policy gradient algorithm with value function baseline in the cell below. The use of baseline is optional.

In [7]:
%%wandb
#remove the above line if you do not want to see wandb plots in your notebook. You can always see them on the wandb website.

#You can change the code in this cell anyway you want 
#However, just make sure per epsiode reward during the run of this algorithm is being recorded in list rrecord 
#and logged on wandb as in the last few lines.

# parameter initializations (you can change any of these)
alpha =1e-2  # learning rate for PG
beta = 1e-2  # learning rate for baseline
numtrajs = 64  # num of trajecories from the current policy to collect in each iteration
iterations = 50  # total num of iterations
envname = "CartPole-v0"  # environment name
gamma = .8  # discount

# initialize environment
env = gym.make(envname)
obssize = env.observation_space.low.size
actsize = env.action_space.n

# initialize networks
actor = Policy(obssize, actsize, alpha)  # policy initialization: IMPORTANT: this is the policy you will be scored on
baseline = ValueFunction(obssize, beta)  # baseline initialization

#To record training reward for logging and plotting purposes
rrecord = []

# main iteration
for ite in range(iterations): 
    
    # To record traectories generated from current policy
    OBS = []  # observations
    ACTS = []  # actions
    ADS = []  # advantages (to compute policy gradient)
    VAL = []  # Monte carlo value predictions (to compute baseline, and policy gradient)

    for num in range(numtrajs):
        # To keep a record of states actions and reward for each episode
        obss = []  # states
        acts = []   # actions
        rews = []  # instant rewards

        obs = env.reset()
        done = False

        # TODO: run one episode using the current policy "actor" 
        while not done:
            prob = actor.compute_prob(np.expand_dims(obs, 0))
            prob /= np.sum(prob)
            action = np.asscalar(np.random.choice(actsize, p=prob.flatten(), size=1))
            obss.append(obs)
            obs, reward, done, info = env.step(action)
        # TODO: record all observations (states, actions, rewards) from the epsiode in  obss, acts, rews
            acts.append(action)
            rews.append(reward)
        #Below is for logging training performance
        rrecord.append(np.sum(rews))
        
        # TODO:  Use discounted_rewards function to compute \hat{V}s/\hat{Q}s  from instant rewards in rews
        V_hat = discounted_rewards(rews, gamma)
#         V_hat = 
        
        # TODO: record the computed \hat{V}s in VAL, states obss in OBS, 
        #       and actions acts in ACTS, for batch update
        VAL.append(np.array(V_hat))
        OBS.append(np.vstack(obss))
        ACTS.append(np.array(acts))
        
        
    # AFTER collecting numtrajs trajectories:
    OBS, VAL, ACTS = np.vstack(OBS), np.hstack(VAL), np.hstack(ACTS)
    
    #1. TODO: train baseline
    """
        Use the batch (OBS, VAL) of states and value predictions as targets to train baseline. 
        Use baseline.train : note that this takes as input numpy array, so you may have to convert 
        lists into numpy array using np.array()
    """
    baseline.train(OBS, VAL.reshape(-1, 1))
    
    # 2.TODO: Update the policy
    """ 
        Compute baselines: use basline.compute_values for states in the batch OBS
        Compute advantages ADS using VAL and computed baselines
        Update policy using actor.train using OBS, ACTS and ADS
    """
    ADS = VAL - baseline.compute_values(OBS).reshape(-1,)
    actor.train(OBS, ACTS, ADS)
     
    #printing moving averages for smoothed visualization. 
    #Do not change below: this assume you recorded the sum of rewards in each episide in the list rrecord
    fixedWindow=100
    movingAverage=0
    if len(rrecord) >= fixedWindow:
        movingAverage=np.mean(rrecord[len(rrecord)-fixedWindow:len(rrecord)-1])
        
    #wandb logging
    wandb.log({ "training reward" : rrecord[-1], "training reward moving average" : movingAverage})

/Users/syeehyn/opt/anaconda3/envs/ieor4575/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


Finally, we evaluate the performance of the trained agent. We will evaluate the performance of the trained policy. The evaluation will be run for 100 epsiodes and print out the average performance across these episodes. Please **do not** change the code below.

In [8]:
# DO NOT CHANGE CODE HERE

### DO NOT CHANGE
def evaluate(policy, env, episodes):
    # main iteration
    score = 0
    for episode in range(episodes):
        
        obs = env.reset()
        done = False
        rsum = 0
        
        while not done:
            
            p = policy.compute_prob(np.expand_dims(obs,0)).ravel()
            p /= np.sum(p)
            action = np.asscalar(np.random.choice(np.arange(2), size=1, p=p))

            # env stepping forward
            newobs, r, done, _ = env.step(action)

            # update data
            rsum += r
            obs = newobs        

        
        wandb.log({"eval reward" : rsum})
        score +=rsum
    score = score/episodes
        
    return score

In [9]:
# DO NOT CHANGE CODE HERE
# after training, we will evaluate the performance of the learned policy "actor"
# on a target environment
env_test = gym.make(envname)
eval_episodes = 1000
score = evaluate(actor, env_test, eval_episodes)
wandb.run.summary["score"]=score 
print("eval performance of the learned policy: {}".format(score))

/Users/syeehyn/opt/anaconda3/envs/ieor4575/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


eval performance of the learned policy: 199.82


In [10]:
run.finish()

training reward,200.0
training reward moving average,199.12121
_step,1049
_runtime,126
_timestamp,1617644282
eval reward,200.0
score,199.82


training reward,▁▂▂▁▄▂▂▃▅▇▆▇▇▄█████▇█▇██████▇▅█▇█▆▆█████
training reward moving average,▁▂▂▃▃▃▄▄▅▆▆▇▇▇██████████████▇▇▇▇▇▇██████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▃▆▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████
_timestamp,▁▃▆▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████
eval reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
